title: NPFL122, Lecture 4
class: title, langtech, cc-by-nc-sa
# N-step Methods, Function Approximation

## Milan Straka

### November 05, 2018

---
section: Refresh
style: pre { font-size: 65% }
# Monte Carlo Assignment

```python
Q = np.zeros([env.states, env.actions])
C = np.zeros([env.states, env.actions])
epsilon = args.epsilon
evaluating = False

while True:
    # Perform episode
    state = env.reset(evaluating)
    states, actions, rewards = [], [], []
    while True:
        if evaluating or np.random.uniform() > epsilong: action = np.argmax(Q[state])
        else:                                            action = np.random.randint(env.actions)

        next_state, reward, done, _ = env.step(action)

        states.append(state)
        actions.append(action)
        rewards.append(reward)

        state = next_state
        if done:
            break

    if env.episode >= args.episodes:
        evaluating = True
```

---
style: pre { font-size: 65% }
# Monte Carlo Assignment

```python
    if not evaluating:
        # Sum discounted rewards
        for i in reversed(range(len(rewards) - 1)):
            rewards[i] += args.gamma * rewards[i + 1]

        # update Q and C
        for i in range(len(rewards)):
            C[states[i]][actions[i]] += 1
            Q[states[i]][actions[i]] += 1 / C[states[i]][actions[i]] * (rewards[i] - Q[states[i]][actions[i]])

        if args.epsilon_final:
            epsilon = np.exp(np.interp(env.episode + 1,
                                       [0, args.episodes],
                                       [np.log(args.epsilon), np.log(args.epsilon_final)]))
```

---
# Sarsa and Q-learning

A straightforward application to the temporal-difference policy evaluation
is Sarsa algorithm, which after generating $S_t, A_t, R_{t+1}, S_{t+1}, A_{t+1}$
computes
$$q(S_t, A_t) ← q(S_t, A_t) + α\left[R_{t+1} + γ q(S_{t+1}, A_{t+1}) -q(S_t, A_t)\right].$$


Q-learning was an important early breakthrough in reinforcement learning (Watkins, 1989).

$$q(S_t, A_t) ← q(S_t, A_t) + α\left[R_{t+1} +  γ \max_a q(S_{t+1}, a) -q(S_t, A_t)\right].$$

---
# Refresh – Q-learning versus Sarsa

![w=70%,h=center](../03/cliff_walking.pdf)

 ~
# Refresh – Q-learning versus Sarsa
![w=40%,h=center](../03/cliff_walking.pdf)
![w=45%,h=center](../03/cliff_walking_learning.pdf)

---
# Refresh – Off-policy Prediction

Given an initial state $S_t$ and an episode $A_t, S_{t+1}, A_{t+1}, …, S_T$,
the probability of this episode under a policy $π$ is
$$∏_{k=t}^{T-1} π(A_k | S_k) p(S_{k+1} | S_k, A_k).$$


Therefore, the relative probability of a trajectory under the target and
behaviour policies is
$$ρ_t ≝ \frac{∏_{k=t}^{T-1} π(A_k | S_k) p(S_{k+1} | S_k, A_k)}{∏_{k=t}^{T-1} b(A_k | S_k) p(S_{k+1} | S_k, A_k)}
      = ∏_{k=t}^{T-1} \frac{π(A_k | S_k)}{b(A_k | S_k)}.$$


Therefore, if $G_t$ is a return of episode generated according to $b$, we can
estimate
$$v_π(S_t) = 𝔼_b[ρ_t G_t].$$

---
# Refresh – Off-policy Monte Carlo Prediction

Let $𝓣(s)$ be a set of times when we visited state $s$. Given episodes sampled
according to $b$, we can estimate
$$v_π(s) = \frac{∑_{t∈𝓣(s)} ρ_t G_t}{|𝓣(s)|}.$$


Such simple average is called _ordinary importance sampling_. It is unbiased, but
can have very high variance.


An alternative is _weighted importance sampling_, where we compute weighted
average as
$$v_π(s) = \frac{∑_{t∈𝓣(s)} ρ_t G_t}{∑_{t∈𝓣(s)} ρ_t}.$$


Weighted importance sampling is biased (with bias asymptotically converging to
zero), but usually has smaller variance.

---
# Refresh – Off-policy Monte Carlo Prediction

![w=80%,h=center](../03/importance_sampling.pdf)

Comparison of ordinary and weighted importance sampling on Blackjack. Given
a state with sum of player's cards 13 and a usable ace, we estimate target
policy of sticking only with a sum of 20 and 21, using uniform behaviour policy.

---
# Refresh – Expected Sarsa

The action $A_{t+1}$ is a source of variance, moving only _in expectation_.


We could improve the algorithm by considering all actions proportionally to their
policy probability, obtaining Expected Sarsa algorithm:
$$\begin{aligned}
  q(S_t, A_t) &← q(S_t, A_t) + α\left[R_{t+1} + γ 𝔼_π q(S_{t+1}, a) - q(S_t, A_t)\right]\\
              &← q(S_t, A_t) + α\left[R_{t+1} + γ ∑\nolimits_a π(a|S_{t+1}) q(S_{t+1}, a) - q(S_t, A_t)\right].
\end{aligned}$$


Compared to Sarsa, the expectation removes a source of variance and therefore
usually performs better. However, the complexity of the algorithm increases and
becomes dependent on number of actions $|𝓐|$.


Note that Expected Sarsa is also an off-policy algorithm, allowing the behaviour
policy $b$ and target policy $π$ to differ.


Especially, if $π$ is a greedy policy with respect to current value function,
Expected Sarsa simplifies to Q-learning.

---
section: Double Q
# Q-learning and Maximization Bias

Because behaviour policy in Q-learning is $ε$-greedy variant of the target
policy, the same samples (up to $ε$-greedy) determine both the maximizing action
and estimate its value.


![w=75%,h=center](double_q_learning_example.pdf)

---
section: Double Q
# Double Q-learning

![w=80%,h=center](double_q_learning.pdf)

---
section: $n$-step Methods
# $n$-step Methods

![w=40%,f=right](nstep_td.pdf)

Full return is
$$G_t = ∑_{k=t}^∞ R_{k+1},$$
one-step return is
$$G_{t:t+1} = R_{t+1} + γ V(S_{t+1}).$$


We can generalize both into $n$-step returns:
$$G_{t:t+n} ≝ \left(∑_{k=t}^{t+n-1} γ^{k-t} R_{k+1}\right) + γ^n V(S_{t+n}).$$
with $G_{t:t+n} ≝ G_t$ if $t+n ≥ T$.

---
# $n$-step Methods

A natural update rule is
$$V(S_t) ← V(S_t) + α\left[G_{t:t+n} - V(S_t)\right].$$


![w=55%,h=center](nstep_td_prediction.pdf)

---
# $n$-step Methods Example

Using the random walk example, but with 19 states instead of 5,
![w=50%,h=center](../03/td_mc_comparison_example.pdf)

we obtain the following comparison of different values of $n$:
![w=50%,h=center](nstep_td_performance.pdf)

---
section: $n$-step Sarsa
# $n$-step Sarsa

Defining the $n$-step return to utilize action-value function as
$$G_{t:t+n} ≝ \left(∑_{k=t}^{t+n-1} γ^{k-t} R_{k+1}\right) + γ^n Q(S_{t+n}, A_{t+n})$$
with $G_{t:t+n} ≝ G_t$ if $t+n ≥ T$,

we get the following straightforward
algorithm:
$$Q(S_t, A_t) ← Q(S_t, A_t) + α\left[G_{t:t+n} - Q(S_t, A_t)\right].$$


![w=70%,h=center](nstep_sarsa_example.pdf)

---
# $n$-step Sarsa Algorithm

![w=60%,h=center](nstep_sarsa_algorithm.pdf)

---
# Off-policy $n$-step Sarsa

Recall the relative probability of a trajectory under the target and behaviour policies,
which we now generalize as
$$ρ_{t:t+n} ≝ ∏_{k=t}^{\min(t+n, T-1)} \frac{π(A_k | S_k)}{b(A_k | S_k)}.$$


Then a simple off-policy $n$-step TD can be computed as
$$V(S_t) ← V(S_t) + αρ_{t:t+n-1}\left[G_{t:t+n} - V(S_t)\right].$$


Similarly, $n$-step Sarsa becomes
$$Q(S_t, A_t) ← Q(S_t, A_t) + αρ_{\boldsymbol{t+1}:\boldsymbol{t+n}}\left[G_{t:t+n} - Q(S_t, A_t)\right].$$

---
# Off-policy $n$-step Sarsa

![w=60%,h=center](off_policy_nstep_sarsa.pdf)

---
section: Tree Backup
# Off-policy $n$-step Without Importance Sampling

![w=30%,h=center](off_policy_nstep_algorithms.pdf)

Q-learning and Expected Sarsa can learn off-policy without importance sampling.


To generalize to $n$-step off-policy method, we must compute expectations
over actions in each step of $n$-step update. However, we have not obtained
a return for the non-sampled actions.


Luckily, we can estimate their values by using the current action-value
function.

---
# Off-policy $n$-step Without Importance Sampling

![w=10%,f=right](tree_backup_example.pdf)


We now derive the $n$-step reward, starting from one-step:
$$G_{t:t+1} ≝ R_{t+1} + ∑\nolimits_a π(a|S_{t+1}) Q(S_{t+1}, a).$$


For two-step, we get:
$$G_{t:t+2} ≝ R_{t+1} + γ∑\nolimits_{a≠A_{t+1}} π(a|S_{t+1}) Q(S_{t+1}, a) + γπ(A_{t+1}|S_{t+1})G_{t+1:t+2}.$$


Therefore, we can generalize to:
$$G_{t:t+n} ≝ R_{t+1} + γ∑\nolimits_{a≠A_{t+1}} π(a|S_{t+1}) Q(S_{t+1}, a) + γπ(A_{t+1}|S_{t+1})G_{t+1:t+n}.$$

---
# Off-policy $n$-step Without Importance Sampling

![w=55%,h=center](tree_backup_algorithm.pdf)

---
section: Function Approximation
# Function Approximation

We will approximate value function $v$ and/or state-value function $q$, choosing
from a family of functions parametrized by a weight vector $→w∈ℝ^d$.


We will denote the approximations as
$$\begin{gathered}
  v̂(s, →w),\\
  q̂(s, a, →w).
\end{gathered}$$



Weights are usually shared among states. Therefore, we need to define state
distribution $μ(s)$ to allow an objective for finding the best function approximation.


The state distribution $μ(s)$ gives rise to a natural objective function called
_Mean Squared Value Error_, denoted $\overline{VE}$:
$$\overline{VE}(→w) ≝ ∑_{s∈𝓢} μ(s) \left[v_π(s) - v̂(s, →w)\right]^2.$$

---
# Function Approximation

For on-policy algorithms, $μ$ is usually on-policy distribution. That is the
stationary distribution under $π$ for continuous tasks, and for the episodic
case it is defined as
$$\begin{aligned}
  η(s) &= h(s) + ∑_{s'}η(s')∑_a π(a|s') p(s|s', a),\\
  μ(s) &= \frac{η(s)}{∑_{s'} η(s')},
\end{aligned}$$
where $h(s)$ is a probability that an episodes starts in state $s$.

---
# Gradient and Semi-Gradient Methods

The functional approximation (i.e., the weight vector $→w$) is usually optimized
using gradient methods, for example as
$$\begin{aligned}
  →w_{t+1} &← →w_t - \frac{1}{2} α ∇ \left[v_π(S_t) - v̂(S_t, →w_t)\right]^2\\
           &← →w_t + α\left[v_π(S_t) - v̂(S_t, →w_t)\right] ∇ v̂(S_t, →w_t).\\
\end{aligned}$$

As usual, the $v_π(S_t)$ is estimated by a suitable sample. For example in Monte
Carlo methods, we use episodic return $G_t$, and in temporal difference methods,
we employ bootstrapping and use $R_{t+1} + γv̂(S_{t+1}, →w).$

---
# Monte Carlo Gradient Policy Evaluation
![w=100%,v=middle](grad_mc_estimation.pdf)

---
# Linear Methods

A simple special case of function approximation are linear methods, where
$$v̂(→x(s), →w) ≝ →x(s)^T →w = ∑x(s)_i w_i.$$


The $→x(s)$ is a representation of state $s$, which is a vector of the same size
as $→w$. It is sometimes called a _feature vector_.


The SGD update rule then becomes
$$→w_{t+1} ← →w_t + α\left[v_π(S_t) - v̂(→x(S_t), →w_t)\right] →x(S_t).$$

---
# State Aggregation

Simple way of generating a feature vector is _state aggregation_, where several
neighboring states are grouped together.


For example, consider a 1000-state random walk, where transitions go uniformly
randomly to any of 100 neighboring states on the left or on the right. Using
state aggregation, we can partition the 1000 states into 10 groups of 100
states. Monte Carlo policy evaluation then computes the following:

![w=60%,h=center](grad_mc_estimation_example.pdf)

---
# Feature Construction for Linear Methods

Many methods developed in the past:


- polynomials


- Fourier basis


- tile coding


- radial basis functions


But of course, nowadays we use deep neural networks which construct a suitable
feature vector automatically as a latent variable (the last hidden layer).

---
# Tile Coding

![w=100%,mh=90%,v=middle](tile_coding.pdf)


If $t$ overlapping tiles are used, the learning rate is usually normalized as $α/t$.

---
# Tile Coding

For example, on the 1000-state random walk example, the performance of tile
coding surpasses state aggregation:

![w=60%,h=center](tile_coding_performance.pdf)

---
# Asymmetrical Tile Coding

In higher dimensions, the tiles should have asymmetrical offsets, with
a sequence of $(1, 3, 5, …, 2d-1)$ being a good choice.

![w=50%,h=center](tile_coding_asymmetrical.pdf)
